## Transformacion
 En este cuaderno realizamos la transformacion el los datos, usando los csv extraidos creamos un data frame con la informacion de los csv

### .


In [47]:
import os
import numpy as np
import pandas as pd
import requests
from time import sleep
from pprint import pprint
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime

In [48]:
shows = pd.read_csv("01_shows_master.csv")
eps = pd.read_csv("02_episodios_raw.csv")

In [49]:
shows.head(2)

,id,name,status,start_date,end_date,network,country,runtime,rating,rating_count,genres,total_episodes
0,75924,Partner Track,Ended,2022-08-26,NaN,Netflix,US,10,5.5,2,Drama,10
1,75925,La divina gula,To Be Determined,2022-01-19,NaN,Netflix,US,10,0.0,0,Food,6


In [50]:
eps.head(2)

,season,episode,name,air_date,show_id
0,1,1,Material Adverse Change,2022-08-26 12:00:00,75924
1,1,2,Meta-Strategy,2022-08-26 12:00:00,75924


In [51]:
#Convertir fechas
eps["air_date"] = pd.to_datetime(eps["air_date"], errors="coerce")
shows["start_date"] = pd.to_datetime(shows["start_date"], errors="coerce")

In [52]:
eps_min = eps.groupby("show_id", as_index=False)["air_date"].min()
eps_min.rename(columns={"air_date": "episode_date"}, inplace=True)
eps_min["episode_date"] = eps_min["episode_date"].dt.strftime("%Y-%m-%d")

eps_min.head(2)

,show_id,episode_date
0,45375,2014-11-28
1,75924,2022-08-26


In [53]:
eps_min = eps.groupby("show_id", as_index=False)["air_date"].min()
eps_min.rename(columns={"air_date": "episode_date"}, inplace=True)

# Mantener solo la fecha como tipo datetime.date
eps_min["episode_date"] = eps_min["episode_date"].dt.date

eps_min.head(2)

,show_id,episode_date
0,45375,2014-11-28
1,75924,2022-08-26


In [54]:
df = pd.DataFrame(data = shows, columns = ["id", "name", "status", "start_date", "end_date", "network", "country", "runtime", "rating", "rating_count","genres","total_episodes"])

df

,id,name,status,start_date,end_date,network,country,runtime,rating,rating_count,genres,total_episodes
0,75924,Partner Track,Ended,2022-08-26,NaN,Netflix,US,10,5.5,2,Drama,10
1,75925,La divina gula,To Be Determined,2022-01-19,NaN,Netflix,US,10,0.0,0,Food,6
2,75926,Fotobonden,To Be Determined,2020-01-07,NaN,NRK TV,NO,10,0.0,0,Nature,15
3,75927,Thursday Night Football on Prime Video,Running,2022-09-15,NaN,Prime Video,US,10,0.0,0,NaN,63
4,75928,48 часов,Ended,2021-12-25,NaN,Пятый канал,RU,39,0.0,0,NaN,16
...,...,...,...,...,...,...,...,...,...,...,...,...
1137,93995,Restarted Love,Ended,2025-08-22,NaN,Tencent QQ,CN,10,0.0,0,Drama,15
1138,93996,The Miracles,In Development,NaT,NaN,Tencent QQ,CN,10,0.0,0,Drama,0
1139,93997,С новым счастьем!,Running,2025-03-28,NaN,ТВ Центр,RU,10,0.0,0,"Drama, Mystery, Romance",2
1140,93998,Катя и Эф. Куда-Угодно-Дверь,Running,2018-05-31,NaN,КиноПоиск,RU,10,0.0,0,"Adventure, Children",119


In [57]:
eps.head(100)

,season,episode,name,air_date,show_id
0,1,1,Material Adverse Change,2022-08-26 12:00:00,75924
1,1,2,Meta-Strategy,2022-08-26 12:00:00,75924
2,1,3,Change of Venue,2022-08-26 12:00:00,75924
3,1,4,Due Diligence,2022-08-26 12:00:00,75924
4,1,5,Out of Office,2022-08-26 12:00:00,75924
...,...,...,...,...,...
95,1,2,Коробок спичек,2021-12-24 23:25:00,75928
96,1,3,Лекарство от боли,2021-12-25 00:15:00,75928
97,1,4,Беглец,2021-12-25 01:05:00,75928
98,1,5,Ничего личного,2021-12-25 01:55:00,75928


In [59]:
eps_show_id = eps.groupby(by = "show_id", as_index = False).agg({"season" : "max"})
eps_show_id.head()

,show_id,season
0,45375,1
1,75924,1
2,75925,1
3,75926,2
4,75927,4


### Comparar fechas de inicio
Una vez que las columnas están en formato datetime (solo la fecha)
Del dataframe de episodios, hacer un groupby por la columna de id y quedarme con el elemento más pequeño de la columna de fecha (min)
Luego comparar ambas columnas del dataframe nuevo y el dataframe de shows. Quedarse con la más pequeña.

### Total de episodios por año
Del df de episodios, crear la columna año (year) y hacer groupby sobre la columna year y contar cualquier otra columna, puede ser la columna de id_show, puede ser con un value count, si es para una gráfica, lo mejor sería un groupby y ordenar por año

In [60]:
df_unido = pd.merge(left=shows, right=eps_show_id,left_on='id',right_on='show_id',how='left')
df_unido.head()

,id,name,status,start_date,end_date,network,country,runtime,rating,rating_count,genres,total_episodes,show_id,season
0,75924,Partner Track,Ended,2022-08-26,NaN,Netflix,US,10,5.5,2,Drama,10,75924.0,1.0
1,75925,La divina gula,To Be Determined,2022-01-19,NaN,Netflix,US,10,0.0,0,Food,6,75925.0,1.0
2,75926,Fotobonden,To Be Determined,2020-01-07,NaN,NRK TV,NO,10,0.0,0,Nature,15,75926.0,2.0
3,75927,Thursday Night Football on Prime Video,Running,2022-09-15,NaN,Prime Video,US,10,0.0,0,NaN,63,75927.0,4.0
4,75928,48 часов,Ended,2021-12-25,NaN,Пятый канал,RU,39,0.0,0,NaN,16,75928.0,1.0


In [61]:
df_unido[~(df_unido["rating"] == 0)].head()

,id,name,status,start_date,end_date,network,country,runtime,rating,rating_count,genres,total_episodes,show_id,season
0,75924,Partner Track,Ended,2022-08-26,NaN,Netflix,US,10,5.5000,2,Drama,10,75924.0,1.0
6,75930,I Am Shauna Rae,To Be Determined,2022-01-11,NaN,TLC,US,60,10.0000,1,NaN,16,75930.0,2.0
13,75937,RuPaul's Drag Race UK vs The World,Running,2022-01-17,NaN,BBC Three,GB,10,7.0000,1,NaN,14,75937.0,2.0
53,75978,The Dawn of the Witch,Running,2022-04-07,NaN,TBS,JP,24,10.0000,3,"Comedy, Adventure, Fantasy, Anime",12,75978.0,1.0
57,75982,Call of the Night,Running,2022-07-07,NaN,Fuji TV,JP,30,9.6667,6,"Comedy, Fantasy, Anime, Romance",25,75982.0,2.0


In [62]:
# DataFrame de generos por show
df_genres = pd.DataFrame(data = df_unido[["id", "genres"]], columns = ["id", "genres"])

df_genres["genres"] = df_genres["genres"].apply(lambda x : x.split(", ") if not pd.isna(x) else x)

df_genres = df_genres.explode("genres")

df_genres.shape

(1735, 2)

In [63]:
df_genres.value_counts("genres")

genres
Drama              288
Romance            196
Comedy             129
Crime              124
Mystery             59
Thriller            53
Action              48
History             45
Fantasy             43
Anime               38
Adventure           37
Food                33
Sports              29
Travel              22
Children            21
Music               20
Science-Fiction     19
Family              19
Horror              13
Nature              13
War                 12
Supernatural        12
Medical             11
Espionage            5
Legal                5
DIY                  4
Western              3
Adult                1
Interview            1
Politics             1
Lifestyle            1
Literature           1
Name: count, dtype: int64

In [64]:
df_unido.value_counts("rating", normalize = True)*100

rating
0.0000     93.345009
10.0000     2.889667
9.0000      0.875657
8.0000      0.612960
7.0000      0.612960
9.6667      0.350263
6.0000      0.175131
3.0000      0.087566
5.5000      0.087566
5.0000      0.087566
8.3333      0.087566
8.5000      0.087566
9.3333      0.087566
9.1250      0.087566
9.5000      0.087566
9.4000      0.087566
9.7000      0.087566
9.7778      0.087566
9.8333      0.087566
9.8750      0.087566
Name: proportion, dtype: float64

In [65]:
eps.info()

shows.info()

df_unido.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25659 entries, 0 to 25658
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   season    25659 non-null  int64         
 1   episode   25659 non-null  int64         
 2   name      25659 non-null  object        
 3   air_date  25659 non-null  datetime64[ns]
 4   show_id   25659 non-null  int64         
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 1002.4+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1142 entries, 0 to 1141
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              1142 non-null   int64         
 1   name            1142 non-null   object        
 2   status          1142 non-null   object        
 3   start_date      995 non-null    datetime64[ns]
 4   end_date        1 non-null      object        
 5   network         1142 n

In [66]:
#primeros 5
df_unido.head()

,id,name,status,start_date,end_date,network,country,runtime,rating,rating_count,genres,total_episodes,show_id,season
0,75924,Partner Track,Ended,2022-08-26,NaN,Netflix,US,10,5.5,2,Drama,10,75924.0,1.0
1,75925,La divina gula,To Be Determined,2022-01-19,NaN,Netflix,US,10,0.0,0,Food,6,75925.0,1.0
2,75926,Fotobonden,To Be Determined,2020-01-07,NaN,NRK TV,NO,10,0.0,0,Nature,15,75926.0,2.0
3,75927,Thursday Night Football on Prime Video,Running,2022-09-15,NaN,Prime Video,US,10,0.0,0,NaN,63,75927.0,4.0
4,75928,48 часов,Ended,2021-12-25,NaN,Пятый канал,RU,39,0.0,0,NaN,16,75928.0,1.0


In [67]:
#ultimos 5
df_unido.tail()

,id,name,status,start_date,end_date,network,country,runtime,rating,rating_count,genres,total_episodes,show_id,season
1137,93995,Restarted Love,Ended,2025-08-22,NaN,Tencent QQ,CN,10,0.0,0,Drama,15,93995.0,1.0
1138,93996,The Miracles,In Development,NaT,NaN,Tencent QQ,CN,10,0.0,0,Drama,0,NaN,NaN
1139,93997,С новым счастьем!,Running,2025-03-28,NaN,ТВ Центр,RU,10,0.0,0,"Drama, Mystery, Romance",2,93997.0,1.0
1140,93998,Катя и Эф. Куда-Угодно-Дверь,Running,2018-05-31,NaN,КиноПоиск,RU,10,0.0,0,"Adventure, Children",119,93998.0,3.0
1141,93999,Irreplaceable,In Development,NaT,NaN,Youku,CN,10,0.0,0,Romance,0,NaN,NaN


In [68]:
# Calcula la media, la mediana y la desviación estandar para cada columna numérica.

print(f'Columna: runtime, Media: {df_unido["runtime"].mean()}, Median: {df_unido["runtime"].median()}, std: {df_unido["runtime"].std()}')
print(f'Columna: rating, Media: {df_unido["rating"].mean()}, Median: {df_unido["rating"].median()}, std: {df_unido["rating"].std()}')
print(f'Columna: rating_count, Media: {df_unido["rating_count"].mean()}, Median: {df_unido["rating_count"].median()}, std: {df_unido["rating_count"].std()}')
print(f'Columna: total_episodes, Media: {df_unido["total_episodes"].mean()}, Median: {df_unido["total_episodes"].median()}, std: {df_unido["total_episodes"].std()}')
print(f'Columna: season, Media: {df_unido["season"].mean()}, Median: {df_unido["season"].median()}, std: {df_unido["season"].std()}')


Columna: runtime, Media: 28.6584938704028, Median: 10.0, std: 30.768406525988745
Columna: rating, Media: 0.5976746935201401, Median: 0.0, std: 2.2696557110368603
Columna: rating_count, Media: 0.14798598949211908, Median: 0.0, std: 0.776512331086509
Columna: total_episodes, Media: 22.46847635726795, Median: 8.0, std: 62.235990086171235
Columna: season, Media: 92.455, Median: 1.0, std: 419.4589798905355


In [69]:
columnas_numericas = ["runtime", "rating", "rating_count", "total_episodes", "season"]

df_unido[columnas_numericas].mean()

runtime           28.658494
rating             0.597675
rating_count       0.147986
total_episodes    22.468476
season            92.455000
dtype: float64

In [70]:
df_unido[columnas_numericas].median()

runtime           10.0
rating             0.0
rating_count       0.0
total_episodes     8.0
season             1.0
dtype: float64

In [71]:
df_unido[columnas_numericas].std()

runtime            30.768407
rating              2.269656
rating_count        0.776512
total_episodes     62.235990
season            419.458980
dtype: float64

In [72]:
len(df_unido)

1142

In [73]:
# mayor runtime
df_unido.sort_values("runtime", ascending = False).head(1)

,id,name,status,start_date,end_date,network,country,runtime,rating,rating_count,genres,total_episodes,show_id,season
1058,93409,CNN's The Fourth in America,Running,2020-07-04,NaN,CNN,US,420,0.0,0,NaN,6,93409.0,2025.0


In [74]:
# menor runtime
df_unido.sort_values("runtime").head(1)

,id,name,status,start_date,end_date,network,country,runtime,rating,rating_count,genres,total_episodes,show_id,season
230,76608,Irodorimidori,Ended,2022-01-05,NaN,Tokyo MX,JP,3,0.0,0,Anime,8,76608.0,1.0


In [75]:
# mayor rating
df_unido.sort_values("rating", ascending = False).head(1)

,id,name,status,start_date,end_date,network,country,runtime,rating,rating_count,genres,total_episodes,show_id,season
159,76534,NCIS: Sydney,Running,2023-11-10,NaN,Paramount+,AU,10,10.0,1,"Crime, Drama",29,76534.0,3.0


In [76]:
# menor rating
df_unido.sort_values("rating").head(1)

,id,name,status,start_date,end_date,network,country,runtime,rating,rating_count,genres,total_episodes,show_id,season
1138,93996,The Miracles,In Development,NaT,NaN,Tencent QQ,CN,10,0.0,0,Drama,0,NaN,NaN


In [77]:
# tiene mas episodios
df_unido.sort_values("total_episodes", ascending = False).head(1)

,id,name,status,start_date,end_date,network,country,runtime,rating,rating_count,genres,total_episodes,show_id,season
448,82630,Big Brother,Running,2004-09-18,NaN,RTL,HR,75,0.0,0,NaN,886,82630.0,10.0


In [78]:
# tiene menos episodios como saco tos los qeu son 0
df_unido.sort_values("total_episodes").head(1)

,id,name,status,start_date,end_date,network,country,runtime,rating,rating_count,genres,total_episodes,show_id,season
1012,93317,hundkatzemaus,Running,NaT,NaN,VOX,DE,55,0.0,0,NaN,0,NaN,NaN


In [79]:
# tiene mas episodios y rating
df_unido.sort_values(["rating", "total_episodes"], ascending = False).head(1)

,id,name,status,start_date,end_date,network,country,runtime,rating,rating_count,genres,total_episodes,show_id,season
382,80713,Tousouchuu: Great Mission,Running,2023-04-02,NaN,Fuji TV,JP,30,10.0,1,Anime,97,80713.0,1.0


In [80]:
# tiene runtimes y rating
df_unido.sort_values(["runtime", "rating"], ascending = False).head(1)

,id,name,status,start_date,end_date,network,country,runtime,rating,rating_count,genres,total_episodes,show_id,season
1058,93409,CNN's The Fourth in America,Running,2020-07-04,NaN,CNN,US,420,0.0,0,NaN,6,93409.0,2025.0


In [81]:
df_unido.describe()

,id,start_date,runtime,rating,rating_count,total_episodes,show_id,season
count,1142.000000,995,1142.000000,1142.000000,1142.000000,1142.000000,1000.000000,1000.00000
mean,84727.194396,2023-06-13 05:21:17.185929728,28.658494,0.597675,0.147986,22.468476,84330.428000,92.45500
min,45375.000000,1979-12-24 00:00:00,3.000000,0.000000,0.000000,0.000000,45375.000000,1.00000
25%,79306.250000,2022-06-30 00:00:00,10.000000,0.000000,0.000000,4.000000,79291.750000,1.00000
50%,85150.500000,2023-12-04 00:00:00,10.000000,0.000000,0.000000,8.000000,84586.000000,1.00000
75%,89527.750000,2024-10-28 12:00:00,50.000000,0.000000,0.000000,18.000000,89498.250000,1.00000
max,93999.000000,2025-12-07 00:00:00,420.000000,10.000000,10.000000,886.000000,93998.000000,2026.00000
std,6136.501766,NaN,30.768407,2.269656,0.776512,62.235990,6069.971072,419.45898


In [82]:
df_unido["genres"].value_counts()

genres
Drama, Romance            64
Drama                     52
Romance                   46
Crime                     39
Comedy                    33
                          ..
Crime, Action              1
Drama, Fantasy             1
Drama, Thriller, War       1
Action, Thriller           1
Comedy, Fantasy, Anime     1
Name: count, Length: 190, dtype: int64

In [83]:
df_unido["network"].value_counts()


network
Netflix          67
Tencent QQ       51
Prime Video      28
iQIYI            27
YouTube          26
                 ..
Now That's Tv     1
TG4 Player        1
КиноПоиск         1
OCS Max           1
W9                1
Name: count, Length: 371, dtype: int64

In [84]:
df_unido["country"].value_counts()

country
US    396
GB    136
CN    118
RU     84
JP     52
TH     36
KR     36
BE     30
AU     24
FR     24
FI     23
CA     22
DE     19
IN     14
ES     14
NO     13
IE     13
SE     12
NL      8
TR      7
HU      7
PT      6
TW      6
PL      5
IT      5
IL      4
UA      4
NZ      3
DK      3
GR      2
BR      2
AT      2
MY      2
HK      1
IS      1
AE      1
BG      1
EG      1
CL      1
HR      1
ZA      1
EE      1
AM      1
Name: count, dtype: int64

In [85]:
df_unido.head()

,id,name,status,start_date,end_date,network,country,runtime,rating,rating_count,genres,total_episodes,show_id,season
0,75924,Partner Track,Ended,2022-08-26,NaN,Netflix,US,10,5.5,2,Drama,10,75924.0,1.0
1,75925,La divina gula,To Be Determined,2022-01-19,NaN,Netflix,US,10,0.0,0,Food,6,75925.0,1.0
2,75926,Fotobonden,To Be Determined,2020-01-07,NaN,NRK TV,NO,10,0.0,0,Nature,15,75926.0,2.0
3,75927,Thursday Night Football on Prime Video,Running,2022-09-15,NaN,Prime Video,US,10,0.0,0,NaN,63,75927.0,4.0
4,75928,48 часов,Ended,2021-12-25,NaN,Пятый канал,RU,39,0.0,0,NaN,16,75928.0,1.0
